In [11]:
import pandas as pd
import re
import os
import logging
from pathlib import Path

# --- Setup basic logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [22]:
import pandas as pd
import re
import os
import logging
from pathlib import Path

# --- Setup basic logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def run_grade_update():
    """
    Finds grade files, preserves special rows, updates only student scores,
    and saves a structurally identical, truncated gradebook.
    """
    # Clean up and save
    for p in Path.cwd().glob('updated_*'):
        if p.is_file(): p.unlink()
    
    # --- 1. File Discovery ---
    try:
        all_files_in_dir = os.listdir()
        bc_pattern = '.*_Grades-CIVENG_93.csv'
        ic_pattern = 'iClicker_GradesExport_Canvas_.*.csv'
        bc_filename = next(f for f in all_files_in_dir if re.match(bc_pattern, f))
        ic_filename = next(f for f in all_files_in_dir if re.match(ic_pattern, f))
        logging.info(f"✅ Found gradebook: {bc_filename}")
        logging.info(f"✅ Found iClicker file: {ic_filename}")
    except StopIteration:
        logging.error("❌ Critical: Could not find one or both of the required CSV files.")
        return

    # --- 2. Load Data and Separate Rows ---
    try:
        bc = pd.read_csv(bc_filename, dtype=str) # Load all as string to preserve formatting
        ic = pd.read_csv(ic_filename, dtype=str)

        # A robust way to identify non-student rows is by checking for a valid SIS User ID.
        # Student rows have a numeric ID; special rows (like 'Points Possible') do not.
        is_student = pd.to_numeric(bc['SIS User ID'], errors='coerce').notna()
        student_data = bc[is_student].copy()
        special_rows = bc[~is_student].copy()

        # We only need the student data from the iClicker file for calculations
        ic_students = ic[pd.to_numeric(ic['SIS User ID'], errors='coerce').notna()].copy()
        
    except Exception as e:
        logging.error(f"An unexpected error occurred during file loading or processing: {e}")
        return

    # --- 3. Dynamically Identify Columns ---
    ic_poll_col = next((col for col in ic_students.columns if re.match(r'Class \d+ - Poll', col)), None)
    bc_poll_col = next((col for col in student_data.columns if col.startswith(ic_poll_col)), None) if ic_poll_col else None
    bc_total_col = next((col for col in student_data.columns if re.match(r'iClicker \(Total\) \(\d+\)', col)), None)
    
    if not all([ic_poll_col, bc_poll_col, bc_total_col]):
        logging.error("❌ Critical: Could not identify all necessary poll and total columns.")
        return

    # Find the integer index for slicing from the original, full column list
    col_index = bc.columns.get_loc(bc_total_col)
    all_bc_poll_cols = [col for col in student_data.columns if col.startswith('Class') and 'Poll' in col]
    logging.info(f"✔️ Columns identified. Processing up to index {col_index}: '{bc_total_col}'")

    # --- 4. Perform Calculations on Student Data Only ---
    ic_subset = ic_students[['SIS User ID', ic_poll_col]].rename(columns={ic_poll_col: 'New_Score'})
    
    # Merge new scores into the student data
    updated_students = pd.merge(student_data, ic_subset, on='SIS User ID', how='left')

    # Convert score columns to numeric for calculation, filling missing new scores with 0
    updated_students['New_Score'] = pd.to_numeric(updated_students['New_Score'], errors='coerce').fillna(0)
    for col in all_bc_poll_cols:
         updated_students[col] = pd.to_numeric(updated_students[col], errors='coerce').fillna(0)

    logging.info("🔄 Updating poll scores...")
    updated_students[bc_poll_col] = updated_students[[bc_poll_col, 'New_Score']].max(axis=1)

    logging.info("🔄 Recalculating total iClicker score...")
    updated_students[bc_total_col] = updated_students[all_bc_poll_cols].sum(axis=1)

    # --- 5. Reconstruct and Save ---
    # Combine the untouched special rows with the updated student data
    final_df = pd.concat([special_rows, updated_students[bc.columns]], ignore_index=True)
    
    # Ensure original row order is preserved
    final_df = final_df.set_index(pd.Index(bc.index))
    final_df = final_df.reindex(bc.index)

    # Move student test index to the last
    row_to_move = final_df.loc[[2]]
    final_df = pd.concat([final_df.drop(index=2), row_to_move], ignore_index=True)

    # First row to be empty
    final_df.iloc[0]=""

    # Truncate to the desired number of columns
    final_df_truncated = final_df.iloc[:, :col_index + 1]
    logging.info(f"Truncating final CSV to {len(final_df_truncated.columns)} columns.")
    output_filename = f'updated_{bc_filename}'
    final_df_truncated.to_csv(output_filename, encoding='utf-8-sig', index=False)
    
    logging.info(f"\n✨ Done! Updated gradebook saved to '{output_filename}'.")

# --- Main execution block ---
if __name__ == "__main__":
    run_grade_update()

INFO:root:✅ Found gradebook: 2025-09-02T1640_Grades-CIVENG_93.csv
INFO:root:✅ Found iClicker file: iClicker_GradesExport_Canvas_09-02-25.csv
INFO:root:✔️ Columns identified. Processing up to index 6: 'iClicker (Total) (8935426)'
INFO:root:🔄 Updating poll scores...
INFO:root:🔄 Recalculating total iClicker score...
INFO:root:Truncating final CSV to 7 columns.
INFO:root:
✨ Done! Updated gradebook saved to 'updated_2025-09-02T1640_Grades-CIVENG_93.csv'.
